# Traducción Automática de Datos
Traducción de los datasets de inglés a español usando librerías de traducción automática.

In [3]:
import os
import pandas as pd
from tqdm import tqdm
from deep_translator import GoogleTranslator
import logging

# — Configuración de logging —
logging.basicConfig(
    filename='translation.log',
    level=logging.INFO,
    format='%(asctime)s %(levelname)s %(message)s'
)

# — Rutas de entrada y salida —
RAW_DIR       = os.path.join("..", "data", "raw")
TRANSL_DIR    = os.path.join("..", "data", "translated")
os.makedirs(TRANSL_DIR, exist_ok=True)

FAKE_IN       = os.path.join(RAW_DIR, "Fake.csv")
TRUE_IN       = os.path.join(RAW_DIR, "True.csv")
FAKE_OUT      = os.path.join(TRANSL_DIR, "Fake_es.csv")
TRUE_OUT      = os.path.join(TRANSL_DIR, "True_es.csv")

# — Carga y muestreo —
df_fake = pd.read_csv(FAKE_IN).drop_duplicates().sample(n=500, random_state=42)
df_true = pd.read_csv(TRUE_IN).drop_duplicates().sample(n=500, random_state=42)

# — Rellenar nulos y cast a str —
for df in (df_fake, df_true):
    df['title'] = df['title'].fillna('').astype(str)
    df['text']  = df['text'].fillna('').astype(str)

# — Función de traducción con logging y manejo de fallos —
def traducir_columna(df, columna, target_col):
    traducidos = []
    for idx, texto in enumerate(tqdm(df[columna], desc=f"Traduciendo '{columna}'")):
        try:
            traducido = GoogleTranslator(source='auto', target='es').translate(texto)
        except Exception as e:
            logging.warning(f"Fila {idx} columna '{columna}' no traducida: {e}")
            traducido = texto
        traducidos.append(traducido)
        # checkpoint cada 100 iteraciones
        if (idx + 1) % 100 == 0:
            df[target_col] = traducidos + [''] * (len(df) - len(traducidos))
            df.to_csv(FAKE_OUT if df is df_fake else TRUE_OUT, index=False)
    return traducidos

# — Traducir y guardar —
# Fake
df_fake['title_es'] = traducir_columna(df_fake, 'title', 'title_es')
# checkpoint intermedio
df_fake.to_csv(FAKE_OUT, index=False)

df_fake['text_es']  = traducir_columna(df_fake, 'text', 'text_es')
df_fake.to_csv(FAKE_OUT, index=False)
print("✅ Fake traducido y guardado en", FAKE_OUT)

# True
df_true['title_es'] = traducir_columna(df_true, 'title', 'title_es')
df_true.to_csv(TRUE_OUT, index=False)

df_true['text_es']  = traducir_columna(df_true, 'text', 'text_es')
df_true.to_csv(TRUE_OUT, index=False)
print("✅ True traducido y guardado en", TRUE_OUT)


Traduciendo 'text': 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [02:28<00:00,  3.37it/s]


✅ Fake traducido y guardado en ..\data\translated\Fake_es.csv


Traduciendo 'text': 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [02:10<00:00,  3.84it/s]

✅ True traducido y guardado en ..\data\translated\True_es.csv
